In [3]:
from langchain.llms.base import LLM
from typing import List, Optional, Dict, Any
from yandex_gpt import YandexGPTConfigManagerForAPIKey, YandexGPT
from env_vars import *

config_manager = YandexGPTConfigManagerForAPIKey(
    model_type="yandexgpt",
    catalog_id=CATALOGUE_ID,
    api_key=API_KEY
)

class YandexGPTWrapper(LLM):
    # def __init__(self, model_type: str, catalog_id: str, api_key: str):
    #     """
    #     Initializes the YandexGPT wrapper with the given configuration.

    #     Args:
    #         model_type (str): The model type (e.g., "yandexgpt").
    #         catalog_id (str): The catalog ID.
    #         api_key (str): The API key for accessing the Yandex GPT API.
    #     """
    config_manager: YandexGPTConfigManagerForAPIKey 
    yandex_gpt: YandexGPT = YandexGPT(config_manager=config_manager)
        
    @property
    def _llm_type(self) -> str:
        return "yandex_gpt_wrapper_because_the_langchain_yandexgpt_wont_let_authorize"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """
        Calls the YandexGPT model with the given prompt and returns the completion.

        Args:
            prompt (str): The input prompt for the model.
            stop (Optional[List[str]]): Optional stop tokens. Not used in this implementation.

        Returns:
            str: The completion text returned by the YandexGPT model.
        """
        messages = [{'role': 'user', 'text': prompt}]
        completion = self.yandex_gpt.get_sync_completion(messages=messages)
        return completion

    @property
    def identifying_params(self) -> Dict[str, Any]:
        """
        Returns the identifying parameters for the model instance.

        Returns:
            Dict[str, Any]: The parameters used to initialize the model.
        """
        return {
            "model_type": self.config_manager.model_type,
            "catalog_id": self.config_manager.catalog_id,
            "api_key": self.config_manager.api_key
        }

# Usage example:
# Assuming you have the CATALOGUE_ID and API_KEY already defined
yandex_llm = YandexGPTWrapper(config_manager=config_manager)
# response = yandex_llm.invoke("Я дам тебе вопрос из 'Что? Где? Когда?' и ответ к нему. Тебе нужно сымитировать рассуждения знатоков. Их диалог. Выдели важные подсказки из вопроса, приводи к ним ассоциации. Используй в том числе идеи из поп-культуры мифологии литературы и мемов и популярные образы. Рассуждай вслух, долго и по шагам и в конце приди к ответу. Бог на одном юмористическом рисунке несколько раз называет ЕГО “хорошим мальчиком”. Назовите ЕГО. Ответ, к которому тебе нужно прийти: Цербер.")
# print(response)


In [4]:
import pandas as pd
from langchain.chains import LLMChain
from langchain_community.llms import YandexGPT
from langchain_core.prompts import PromptTemplate
from time import sleep

from tqdm import tqdm

tqdm.pandas()

In [28]:
df = pd.read_csv("test_res_with_bin.csv").head(40)

In [29]:
prompt_str = '''
Ты профессиональный игрок в Что? Где? Когда? Ты пытаешься найти ответ на вопрос "Что? Где? Когда?". Для этого ты 1) сперва должен обратить внимание на важные детали вопроса и потенциальные подсказки. 
2) После этого ты пытаешься рассуждать на основании этих подсказок и генерировать идеи и ассоциации, культурные, литературные и исторические и другие. 

3)Потом ты начинаешь строить логические связи и генерировать идеи ответа на основе идей и подсказок.

4) Потом ты выбираешь наиболее правдоподобную и проверяешь, не противоречит ли ответ чему-то в вопросе.
Если ответ противоречит, то он неправильный и ты пишешь в конце #НЕПРАВИЛЬНО#

5) Если ответ не противоречит, ты в самом конце рассуждений пишешь его в таком формате: #ТВОЙ ОТВЕТ#

___________________
ПРИМЕР:
“В предпоследнем эпизоде сериала “Го́тэм” ОНА на некоторое время зависает на фоне луны. Назовите ЕЁ двумя словами.

1 - 2) Я вижу, в вопросе неспроста могут быть: 
эпизод предпоследний. Может это что-то связанное с окончанием или какими-то событиями, которые могут произойти после конца сериала?
Готэм. Это сериал из вселенной DC, Готэм ассоциируется с Бэтменом. Может это что-то про Бэтмена? Хотя сериал это приквел, может что-то связанно с этим?
Зависает на фоне луны. Луна похожа на Бэт-сигнал? 
ОНА - это женский род, это слово женского рода
двумя словами. Нужно найти ответ, состоящий из двух слов.

3) Это что-то про Бэтмена и похоже на бэт-сигнал. Может женщина-кошка? Но она никак не связана с луной, нужно думать дальше. Может летучая мышь?
4)О, на бэт-сигнале как раз изображена летучая мышь! Когда она зависает на фоне луны, мы получаем намек на бэт-сигнал. Как раз сериал это приквел к историям о бэтмене. И то, что это конец сериала, намекает на то, что скоро появится Бэтмен!

5) Нужно проверить, что ответ сходится с вопросом: 
последний эпизод намекает на будущую роль бэтмена
готэм намекает на бэтмена
летучая мышь на фоне луны напоминает бэт сигнал
летучая мышь женского рода
летучая мышь – два слова. Все сходится!

Я думаю, что мне удалось достичь верного ответа.
#ЛЕТУЧАЯ МЫШЬ#
“
____________________

Проделай то же самое со следующим вопросом:

ВОПРОС: {q}

НЕ ПЫТАЙСЯ НАЙТИ ОТВЕТ СРАЗУ.
ПРОЙДИ ВСЕ 5 ШАГОВ.
'''


prompt = PromptTemplate.from_template(prompt_str)

In [30]:
def func(row):
        # llm_chain = {"q": lambda _: row['q'], "a": lambda _: row['a'], "pred":lambda _:  row['predict']} | prompt | yandex_llm
        # llm_chain = prompt | yandex_llm
        llm_chain = {"q": lambda _: row['q']} | prompt | yandex_llm
        # print(llm_chain)
        sleep(2)
        return llm_chain.invoke("")
        # return ""

In [31]:
df.iloc[0]

Unnamed: 0.1                                                     485
Unnamed: 0                                                       492
q                  Автор детективов Иэн Рэнкин увлекается неким з...
a                  собирать пазл, собирать мозаику, складывать па...
predict                                        Раскладывать пасьянс.
f1_score                                                         0.0
exact_match                                                        0
answer_checking    #НЕВЕРНО\n\nИэн Рэнкин увлекается собиранием п...
correct                                                        False
Name: 0, dtype: object

In [32]:
# res = func(df.iloc[0])
# res

In [33]:
len(df)

40

In [34]:
df['thoughts'] = df.progress_apply(func,axis=1)

  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [07:56<00:00, 11.92s/it]


In [35]:
# print(res)

In [36]:
df

,Unnamed: 0.1,Unnamed: 0,q,a,predict,f1_score,exact_match,answer_checking,correct,thoughts
0,485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па...",Раскладывать пасьянс.,0.000000,0,#НЕВЕРНО\n\nИэн Рэнкин увлекается собиранием п...,False,**1–2.** Автор детективов Иэн Рэнкин увлекаетс...
1,817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник",расстоянии,0.000000,0,#НЕВЕРНО.\n\nКоманда ответила на вопрос неполн...,False,**Шаг 1.** В вопросе есть такие детали и подск...
2,30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт",Космонавт,0.666667,0,#ВЕРНО\nОтвет команды «Космонавт» можно зачест...,True,"Конечно, я могу помочь вам с этим вопросом. Во..."
3,551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь,Дон Кихот.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды нельзя зачесть, поск...",False,"Конечно, я могу попытаться найти ответ на этот..."
4,96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра",Надпись.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды не подходит, посколь...",False,"Хорошо, давайте попробуем ответить на этот воп..."
5,244,249,Во время холодной войны американку ДжУдит КОпл...,"роман,",Роман.,1.000000,1,#ВЕРНО\nРассуждения:\nОтвет команды «Роман» мо...,True,1 - 2) В этом вопросе могут быть важны следующ...
6,559,568,В произведении Романа Сефа стадо ИХ исчезает н...,барашки,Овечки.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды не подходит, потому ...",False,"1 - 2) В вопросе упоминается стадо, которое ис..."
7,352,359,Однажды корабль «Баттерфляй» сильно пострадал ...,"кит,",Баттерфляй.,0.000000,0,"#НЕВЕРНО\n\nРассуждения: в вопросе говорится, ...",False,**1–2.** В вопросе есть такие детали:\n* кораб...
8,465,472,Журналистка и феминистка Рут Хейл жила в начал...,девичья фамилия,Девичья фамилия,1.000000,1,#ВЕРНО\n\nОтвет команды совпадает с правильным...,True,"Конечно, я могу помочь вам с вопросом о Рут Хе..."
9,544,552,Рядом с одной ветеринарной клиникой растёт опр...,кот,собака,0.000000,0,#НЕВЕРНО\n\n**Рассуждения**: на логотипе ветер...,False,1 - 2) В вопросе упоминается определённое круп...


In [37]:
prompt_extraction = '''
Ты умеешь играть в "Что? Где? Когда?". Ты должен посмотреть на вопрос и на рассуждения знатока ниже и выделить из них ответ. И ответить только ответ. Ничего больше. Никаких рассуждений кроме этого. 

Вопрос: {q}

Рассуждения: {th}

ВЕРНИ ТОЛЬКО ОТВЕТ ЗНАТОКА БЕЗ ЗНАКОВ ПРЕПИНАНИЯ.
'''

prompt_extr = PromptTemplate.from_template(prompt_extraction)

In [38]:
def func(row):
        # llm_chain = {"q": lambda _: row['q'], "a": lambda _: row['a'], "pred":lambda _:  row['predict']} | prompt | yandex_llm
        # llm_chain = prompt | yandex_llm
        llm_chain = {"q": lambda _: row['q'], "th": lambda _: row['thoughts']} | prompt_extr | yandex_llm
        # print(llm_chain)
        sleep(2)
        return llm_chain.invoke("")
        # return ""

In [39]:
res = func(df.iloc[0])
res

'Сортировка информации'

In [40]:
df1 = df
df1['answer_cot'] = df.progress_apply(func, axis=1)

100%|██████████| 40/40 [02:21<00:00,  3.54s/it]


In [41]:
df1

,Unnamed: 0.1,Unnamed: 0,q,a,predict,f1_score,exact_match,answer_checking,correct,thoughts,answer_cot
0,485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па...",Раскладывать пасьянс.,0.000000,0,#НЕВЕРНО\n\nИэн Рэнкин увлекается собиранием п...,False,**1–2.** Автор детективов Иэн Рэнкин увлекаетс...,Сортировка информации
1,817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник",расстоянии,0.000000,0,#НЕВЕРНО.\n\nКоманда ответила на вопрос неполн...,False,**Шаг 1.** В вопросе есть такие детали и подск...,Охота
2,30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт",Космонавт,0.666667,0,#ВЕРНО\nОтвет команды «Космонавт» можно зачест...,True,"Конечно, я могу помочь вам с этим вопросом. Во...",Исследователь
3,551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь,Дон Кихот.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды нельзя зачесть, поск...",False,"Конечно, я могу попытаться найти ответ на этот...",Давид и Голиаф
4,96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра",Надпись.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды не подходит, посколь...",False,"Хорошо, давайте попробуем ответить на этот воп...",Рука
5,244,249,Во время холодной войны американку ДжУдит КОпл...,"роман,",Роман.,1.000000,1,#ВЕРНО\nРассуждения:\nОтвет команды «Роман» мо...,True,1 - 2) В этом вопросе могут быть важны следующ...,Роман.
6,559,568,В произведении Романа Сефа стадо ИХ исчезает н...,барашки,Овечки.,0.000000,0,"#НЕВЕРНО\n\nОтвет команды не подходит, потому ...",False,"1 - 2) В вопросе упоминается стадо, которое ис...",Барашки
7,352,359,Однажды корабль «Баттерфляй» сильно пострадал ...,"кит,",Баттерфляй.,0.000000,0,"#НЕВЕРНО\n\nРассуждения: в вопросе говорится, ...",False,**1–2.** В вопросе есть такие детали:\n* кораб...,Капитан.
8,465,472,Журналистка и феминистка Рут Хейл жила в начал...,девичья фамилия,Девичья фамилия,1.000000,1,#ВЕРНО\n\nОтвет команды совпадает с правильным...,True,"Конечно, я могу помочь вам с вопросом о Рут Хе...",Личная свобода
9,544,552,Рядом с одной ветеринарной клиникой растёт опр...,кот,собака,0.000000,0,#НЕВЕРНО\n\n**Рассуждения**: на логотипе ветер...,False,1 - 2) В вопросе упоминается определённое круп...,БЕЛКА
